# Download the dataset for key point

In [11]:
import os
os.environ['PATH'] += os.pathsep + '/usr/local/bin'



In [24]:
!wget "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30" -O 'tennis_court_det_dataset.zip'


--2024-08-31 16:00:54--  https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 2607:f8b0:400b:804::2001, 142.251.41.65
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|2607:f8b0:400b:804::2001|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2004 (2.0K) [text/html]
Saving to: ‘tennis_court_det_dataset.zip’

tennis_court_det_da 100%[===================>]   1.96K  --.-KB/s    in 0s      

2024-08-31 16:00:56 (3.96 MB/s) - ‘tennis_court_det_dataset.zip’ saved [2004/2004]



In [22]:
!ls -lh tennis_court_det_dataset.zip

-rw-r--r--  1 yulinxia  staff   2.0K Aug 31 16:00 tennis_court_det_dataset.zip


In [27]:
# import zipfile

# zip_file = 'keypoints_model.zip'

# try:
#     with zipfile.ZipFile(zip_file, 'r') as z:
#         print("This is a valid zip file.")
#         z.printdir()  
# except zipfile.BadZipFile:
#     print("This is not a valid zip file.")

!unzip keypoints_model.zip


Archive:  keypoints_model.zip
 extracting: keypoints_model_3/data.pkl  
 extracting: keypoints_model_3/byteorder  
 extracting: keypoints_model_3/data/0  
 extracting: keypoints_model_3/data/1  
 extracting: keypoints_model_3/data/10  
 extracting: keypoints_model_3/data/100  
 extracting: keypoints_model_3/data/101  
 extracting: keypoints_model_3/data/102  
 extracting: keypoints_model_3/data/103  
 extracting: keypoints_model_3/data/104  
 extracting: keypoints_model_3/data/105  
 extracting: keypoints_model_3/data/106  
 extracting: keypoints_model_3/data/107  
 extracting: keypoints_model_3/data/108  
 extracting: keypoints_model_3/data/109  
 extracting: keypoints_model_3/data/11  
 extracting: keypoints_model_3/data/110  
 extracting: keypoints_model_3/data/111  
 extracting: keypoints_model_3/data/112  
 extracting: keypoints_model_3/data/113  
 extracting: keypoints_model_3/data/114  
 extracting: keypoints_model_3/data/115  
 extracting: keypoints_model_3/data/116  
 extracti

# Code part started

In [31]:
import torch
import json
import cv2
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create torch dataset

In [32]:
class Keypoints(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224,224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data(index)
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        new_img = self.transforms(img)
        kps = np.array(item['kps']).flatten()

        kps = kps.astype(np.float32)

        kps[::2] * (224.0) / w # adjust the x cordinates
        kps[::2] * (224.0) / h # adjust the y cordinates

        return img, kps

         



In [ ]:
train_dataset = Keypoints("data/images", "data/data_train.json")
valid_dataset = Keypoints("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
 
